In [1]:
!nvidia-smi

Sun Aug 18 11:12:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              42W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [3]:
#DATASET

import glob
import random
import os
import numpy as np

import torch
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

# Normalization parameters for pre-trained PyTorch models
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])


class ImageDataset(Dataset):
    def __init__(self, root, hr_shape):
        hr_height, hr_width = hr_shape
        # Transforms for low resolution images and high resolution images
        self.lr_transform = transforms.Compose(
            [
                transforms.Resize((hr_height // 4, hr_height // 4), Image.BICUBIC),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
            ]
        )
        self.hr_transform = transforms.Compose(
            [
                transforms.Resize((hr_height, hr_height), Image.BICUBIC),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
            ]
        )

        self.files = sorted(glob.glob(root + "/*.*"))

    def __getitem__(self, index):
        img = Image.open(self.files[index % len(self.files)])
        img_lr = self.lr_transform(img)
        img_hr = self.hr_transform(img)

        return {"lr": img_lr, "hr": img_hr}

    def __len__(self):
        return len(self.files)

In [4]:
#SRGAN MODEL

import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision.models import vgg19
import math



class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        # 使用預訓練的 VGG19 網絡作為特徵提取器
        vgg = vgg19(pretrained=True)
        # 提取 VGG19 的特定層作為特徵提取器
        self.feature_extractor = nn.Sequential(*list(vgg.features.children())[:18])
        # 初始化權重
        for module in self.modules():
            if isinstance(module, nn.Conv2d):
                nn.init.kaiming_normal_(module.weight, mode="fan_out", nonlinearity="relu")
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
            elif isinstance(module, nn.BatchNorm2d):
                nn.init.constant_(module.weight, 1)
                nn.init.constant_(module.bias, 0)
            elif isinstance(module, nn.Linear):
                nn.init.normal_(module.weight, 0, 0.01)
                nn.init.constant_(module.bias, 0)


    def forward(self, img):
        return self.feature_extractor(img)


class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(in_features, in_features, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(in_features, 0.8),
            nn.PReLU(),
            nn.Conv2d(in_features, in_features, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(in_features, 0.8),
        )

    def forward(self, x):
        return x + self.conv_block(x)


class GeneratorResNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, n_residual_blocks=16):
        super(GeneratorResNet, self).__init__()

        # First layer
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels, 64, kernel_size=9, stride=1, padding=4), nn.PReLU())

        # Residual blocks
        res_blocks = []
        for _ in range(n_residual_blocks):
            res_blocks.append(ResidualBlock(64))
        self.res_blocks = nn.Sequential(*res_blocks)

        # Second conv layer post residual blocks
        self.conv2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(64, 0.8))

        # Upsampling layers
        upsampling = []
        for out_features in range(2):
            upsampling += [
                # nn.Upsample(scale_factor=2),
                nn.Conv2d(64, 256, 3, 1, 1),
                nn.BatchNorm2d(256),
                nn.PixelShuffle(upscale_factor=2),
                nn.PReLU(),
            ]
        self.upsampling = nn.Sequential(*upsampling)

        # Final output layer
        self.conv3 = nn.Sequential(nn.Conv2d(64, out_channels, kernel_size=9, stride=1, padding=4), nn.Tanh())

    def forward(self, x):
        out1 = self.conv1(x)
        out = self.res_blocks(out1)
        out2 = self.conv2(out)
        out = torch.add(out1, out2)
        out = self.upsampling(out)
        out = self.conv3(out)
        return out


class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()

        self.input_shape = input_shape
        in_channels, in_height, in_width = self.input_shape
        patch_h, patch_w = int(in_height / 2 ** 4), int(in_width / 2 ** 4)
        self.output_shape = (1, patch_h, patch_w)

        def discriminator_block(in_filters, out_filters, first_block=False):
            layers = []
            layers.append(nn.Conv2d(in_filters, out_filters, kernel_size=3, stride=1, padding=1))
            if not first_block:
                layers.append(nn.BatchNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            layers.append(nn.Conv2d(out_filters, out_filters, kernel_size=3, stride=2, padding=1))
            layers.append(nn.BatchNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        layers = []
        in_filters = in_channels
        for i, out_filters in enumerate([64, 128, 256, 512]):
            layers.extend(discriminator_block(in_filters, out_filters, first_block=(i == 0)))
            in_filters = out_filters

        layers.append(nn.Conv2d(out_filters, 1, kernel_size=3, stride=1, padding=1))

        self.model = nn.Sequential(*layers)

    def forward(self, img):
        return self.model(img)

In [5]:
"""
Super-resolution of CelebA using Generative Adversarial Networks.
The dataset can be downloaded from: https://www.dropbox.com/sh/8oqt9vytwxb3s4r/AADIKlz8PR9zr6Y20qbkunrba/Img/img_align_celeba.zip?dl=0
(if not available there see if options are listed at http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html)
Instrustion on running the script:
1. Download the dataset from the provided link
2. Save the folder 'img_align_celeba' to '../../data/'
4. Run the sript using command 'python3 srgan.py'
"""

import argparse
import os
import numpy as np
import math
import itertools
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

from torch.utils.data import DataLoader
from torch.autograd import Variable

#from models001Test import *
#from datasets import *

import torch.nn as nn
import torch.nn.functional as F
import torch

"""
# 初始化生成器和特徵提取器
generator = Generator()
feature_extractor = FeatureExtractor()
"""


os.makedirs("images", exist_ok=True)
os.makedirs("saved_models", exist_ok=True)
opt = {
    "epoch": 0,
    "n_epochs": 200,  #訓練的總 epoch 數。每個 epoch 是完整遍歷一次訓練數據集的過程
    #"dataset_name": "DIV2K_train_HR",
    "batch_size": 4,  #每個批次（batch）中處理的圖像數量。批次大小越大，訓練過程中的內存使用量也越大。
    "lr": 0.0002,   #學習率，控制模型的更新步伐。較小的學習率通常會讓模型更穩定，但收斂速度較慢。
    "b1": 0.5,     #這是 Adam 優化器的兩個超參數。b1 控制一階矩估計的動量，b2 控制二階矩估計的動量。
    "b2": 0.999,    #一般來說，這兩個參數的默認值分別為 0.5 和 0.999，這些值通常適合大多數情況。
    "decay_epoch": 101, #從哪個 epoch 開始，學習率會逐漸衰減。
               #這個值設置為 100 意味著從第 100 個 epoch 開始，學習率會開始減少，這有助於穩定模型並防止過度擬合。
    "n_cpu": 8,
    "hr_height": 256,  #高解析度圖像的高度。
    "hr_width": 256,   #高解析度圖像的寬度。
    "channels": 3,  #圖像的通道數。通常為 3（RGB）或 1（灰度）。
    "sample_interval": 100, #訓練過程中，用於保存和顯示樣本圖像的間隔（以批次為單位）。每當完成一定數量的批次後，模型會生成一些圖像樣本進行保存和可視化。
    "checkpoint_interval": 10 #訓練過程中保存模型權重的間隔（以 epoch 為單位）。如果設置為 1，每個 epoch 都會保存一次模型的權重。
}

# 使用變數 opt 中的值
epoch = opt["epoch"]
n_epochs = opt["n_epochs"]
#dataset_name = opt["dataset_name"]
batch_size = opt["batch_size"]
lr = opt["lr"]
b1 = opt["b1"]
b2 = opt["b2"]
decay_epoch = opt["decay_epoch"]
n_cpu = opt["n_cpu"]
hr_height = opt["hr_height"]
hr_width = opt["hr_width"]
channels = opt["channels"]
sample_interval = opt["sample_interval"]
checkpoint_interval = opt["checkpoint_interval"]

print(opt)

cuda = torch.cuda.is_available()

hr_shape = (hr_height, hr_width)

# Initialize generator and discriminator
generator = GeneratorResNet()
discriminator = Discriminator(input_shape=(channels, *hr_shape))
feature_extractor = FeatureExtractor()

# Set feature extractor to inference mode
feature_extractor.eval()

# Losses
criterion_GAN = torch.nn.MSELoss()
criterion_content = torch.nn.L1Loss()

if cuda:
    generator = generator.cuda()
    discriminator = discriminator.cuda()
    feature_extractor = feature_extractor.cuda()
    criterion_GAN = criterion_GAN.cuda()
    criterion_content = criterion_content.cuda()

if epoch != 0:
    # Load pretrained models
    generator.load_state_dict(torch.load("saved_models/generator_%d.pth"))
    discriminator.load_state_dict(torch.load("saved_models/discriminator_%d.pth"))

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor


dataset_path = "/content/drive/MyDrive/ColabNotebooks/SRGANDataset/DIV2K_train_HR"
print(f"Dataset path: {os.path.abspath(dataset_path)}")
print(f"Is dataset path valid? {os.path.exists(dataset_path)}")


dataloader = DataLoader(
    ImageDataset("/content/drive/MyDrive/ColabNotebooks/SRGANDataset/DIV2K_train_HR", hr_shape=hr_shape),
    batch_size=batch_size,
    shuffle=True,
    num_workers=n_cpu,
)

# 打印數據加載器中的一個批次
for batch in dataloader:
    print("Batch loaded successfully")
    break

# 測試 FeatureExtractor 類
model = FeatureExtractor()
print(model)


"""
#測試路徑是否有路具
import argparse
parser = argparse.ArgumentParser()
opt = parser.parse_args()
dataset_path = "../../data/%s" % opt.dataset_name
print("dataset_path")

#測試路徑是否正確
for batch in dataloader:
    print("Batch loaded successfully")
    break
"""



{'epoch': 0, 'n_epochs': 200, 'batch_size': 4, 'lr': 0.0002, 'b1': 0.5, 'b2': 0.999, 'decay_epoch': 101, 'n_cpu': 8, 'hr_height': 256, 'hr_width': 256, 'channels': 3, 'sample_interval': 100, 'checkpoint_interval': 10}


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:02<00:00, 215MB/s]


Dataset path: /content/drive/MyDrive/ColabNotebooks/SRGANDataset/DIV2K_train_HR
Is dataset path valid? True
Batch loaded successfully
FeatureExtractor(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(

'\n#測試路徑是否有路具\nimport argparse\nparser = argparse.ArgumentParser()\nopt = parser.parse_args()\ndataset_path = "../../data/%s" % opt.dataset_name\nprint("dataset_path")\n\n#測試路徑是否正確\nfor batch in dataloader:\n    print("Batch loaded successfully")\n    break\n'

In [6]:
# ----------
#  Training
# ----------

for epoch in range(epoch, n_epochs):
    for i, imgs in enumerate(dataloader):

        # Configure model input
        imgs_lr = Variable(imgs["lr"].type(Tensor))
        imgs_hr = Variable(imgs["hr"].type(Tensor))

        # Adversarial ground truths
        valid = Variable(Tensor(np.ones((imgs_lr.size(0), *discriminator.output_shape))), requires_grad=False)
        fake = Variable(Tensor(np.zeros((imgs_lr.size(0), *discriminator.output_shape))), requires_grad=False)

        # ------------------
        #  Train Generators
        # ------------------

        optimizer_G.zero_grad()

        # Generate a high resolution image from low resolution input
        gen_hr = generator(imgs_lr)

        # Adversarial loss
        loss_GAN = criterion_GAN(discriminator(gen_hr), valid)

        # Content loss
        gen_features = feature_extractor(gen_hr)
        real_features = feature_extractor(imgs_hr)
        loss_content = criterion_content(gen_features, real_features.detach())

        # Total loss
        loss_G = loss_content + 1e-3 * loss_GAN

        loss_G.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss of real and fake images
        loss_real = criterion_GAN(discriminator(imgs_hr), valid)
        loss_fake = criterion_GAN(discriminator(gen_hr.detach()), fake)

        # Total loss
        loss_D = (loss_real + loss_fake) / 2

        loss_D.backward()
        optimizer_D.step()

        # --------------
        #  Log Progress
        # --------------

        sys.stdout.write(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]\n"
            % (epoch, n_epochs, i, len(dataloader), loss_D.item(), loss_G.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            # Save image grid with upsampled inputs and SRGAN outputs
            imgs_lr = nn.functional.interpolate(imgs_lr, scale_factor=4)
            gen_hr = make_grid(gen_hr, nrow=1, normalize=True)
            imgs_lr = make_grid(imgs_lr, nrow=1, normalize=True)
            img_grid = torch.cat((imgs_lr, gen_hr), -1)
            save_image(img_grid, "images/%d.png" % batches_done, normalize=False)

    if checkpoint_interval != -1 and epoch % checkpoint_interval == 0:
        # Save model checkpoints
        torch.save(generator.state_dict(), "saved_models/generator_%d.pth" % epoch)
        torch.save(discriminator.state_dict(), "saved_models/discriminator_%d.pth" % epoch)

"""
torch.save(generator.state_dict(), "saved_models/generator_manual_save.pth")
torch.save(discriminator.state_dict(), "saved_models/discriminator_manual_save.pth")
"""

if os.path.isfile("saved_models/generator_0.pth"):
    print("Generator model saved successfully.")
else:
    print("Generator model not found.")

<ipython-input-6-8ed2d03d6572>:13: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:78.)
  valid = Variable(Tensor(np.ones((imgs_lr.size(0), *discriminator.output_shape))), requires_grad=False)


串流輸出內容已截斷至最後 5000 行。
[Epoch 175/200] [Batch 1/200] [D loss: 0.000089] [G loss: 0.019117]
[Epoch 175/200] [Batch 2/200] [D loss: 0.000223] [G loss: 0.018032]
[Epoch 175/200] [Batch 3/200] [D loss: 0.000276] [G loss: 0.014747]
[Epoch 175/200] [Batch 4/200] [D loss: 0.000301] [G loss: 0.020845]
[Epoch 175/200] [Batch 5/200] [D loss: 0.000154] [G loss: 0.013306]
[Epoch 175/200] [Batch 6/200] [D loss: 0.000099] [G loss: 0.021710]
[Epoch 175/200] [Batch 7/200] [D loss: 0.000105] [G loss: 0.020298]
[Epoch 175/200] [Batch 8/200] [D loss: 0.000082] [G loss: 0.018429]
[Epoch 175/200] [Batch 9/200] [D loss: 0.000105] [G loss: 0.019605]
[Epoch 175/200] [Batch 10/200] [D loss: 0.000101] [G loss: 0.021966]
[Epoch 175/200] [Batch 11/200] [D loss: 0.000133] [G loss: 0.013132]
[Epoch 175/200] [Batch 12/200] [D loss: 0.000263] [G loss: 0.020966]
[Epoch 175/200] [Batch 13/200] [D loss: 0.000433] [G loss: 0.015889]
[Epoch 175/200] [Batch 14/200] [D loss: 0.000169] [G loss: 0.019224]
[Epoch 175/200] [Batch